#### Imports

In [33]:
import pandas as pd
import numpy as np
import os
from pandas_profiling import ProfileReport

#### Load csv

In [102]:
crimes = pd.read_csv('D:/Universidade-Fcul/2_semestre/TPD/projeto/crimes_merged.csv')
crimes.head()

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01000002,City of London,Violence Against the Person,Harassment,0,2011,9
1,E01000005,City of London,Violence Against the Person,Harassment,0,2014,7
2,E01000001,City of London,Violence Against the Person,Harassment,0,2008,3
3,E01032739,City of London,Violence Against the Person,Harassment,0,2016,3
4,E01000001,City of London,Violence Against the Person,Harassment,0,2011,10


#### Drop and cast columns

In [103]:
crimes = crimes.drop(columns = ['lsoa_code'])
crimes['year'] = crimes['year'].astype(str)
crimes['month'] = crimes['month'].astype(str)
crimes.head()

,borough,major_category,minor_category,value,year,month
0,City of London,Violence Against the Person,Harassment,0,2011,9
1,City of London,Violence Against the Person,Harassment,0,2014,7
2,City of London,Violence Against the Person,Harassment,0,2008,3
3,City of London,Violence Against the Person,Harassment,0,2016,3
4,City of London,Violence Against the Person,Harassment,0,2011,10


In [36]:
# profile = ProfileReport(crimes, title='Crimes Profiling Report', explorative=1, minimal =1)

In [5]:
# profile

#### Interesting info

* **value**: Esta coluna apresenta 2895341 valores = 0. Ou seja temos 2895341 linhas, que correspondem a 0 ocorrencias de um certo crime, num dado ano, mês e localização. Fará então sentido eliminar estas linhas, e considerar que a omissão de um certo crime num dado ano, mês e localização, como a sua não ocorrencia.

![alt text](images/value_profile.PNG "Value Profile")

In [104]:
crimes = crimes.drop(crimes.index[crimes['value'] == 0])
print(crimes.shape[0])
crimes.head()

990907


,borough,major_category,minor_category,value,year,month
16,City of London,Violence Against the Person,Harassment,1,2013,7
40,City of London,Violence Against the Person,Harassment,1,2015,5
44,City of London,Violence Against the Person,Harassment,1,2013,8
55,City of London,Violence Against the Person,Harassment,1,2011,10
90,City of London,Violence Against the Person,Harassment,1,2016,4


* **year**: Na coluna 'year', conseguimos ver que a grande maioria dos dados deste dataframe correspondem a crimes entre 2008-2016. Os dados referentes a 2017 e 2018 são têm um volume menor, pois são dados provenientes de um outro dataframe, com um numero inferior de entradas.

![alt text](images/year.PNG "Value Title")

### Crete tables

#### Crimes tables: Major | Minor | Crime

In [38]:
crimes_df = crimes[['major_category', 'minor_category']].drop_duplicates()
crimes_df

,major_category,minor_category
16,Violence Against the Person,Harassment
101,Theft and Handling,Other Theft
232,Violence Against the Person,Wounding/GBH
289,Violence Against the Person,Common Assault
356,Violence Against the Person,Other violence
436,Drugs,Drug Trafficking
473,Violence Against the Person,Offensive Weapon
500,Other Notifiable Offences,Other Notifiable
555,Theft and Handling,Theft From Shops
636,Robbery,Business Property


#### Major Table

In [39]:
major_category_table = pd.DataFrame(crimes_df['major_category'].unique(), columns=['major_category'])
major_category_table['major_category_id'] = major_category_table.index 
major_category_table.head()

,major_category,major_category_id
0,Violence Against the Person,0
1,Theft and Handling,1
2,Drugs,2
3,Other Notifiable Offences,3
4,Robbery,4


#### Minor Table

In [40]:
minor_category_table = pd.DataFrame(crimes_df['minor_category'].unique(), columns=['minor_category'])
minor_category_table['minor_category_id'] = minor_category_table.index 
minor_category_table.head()

,minor_category,minor_category_id
0,Harassment,0
1,Other Theft,1
2,Wounding/GBH,2
3,Common Assault,3
4,Other violence,4


#### Crimes Table

In [41]:
for r in crimes_df.iterrows():
    crimes_df.at[r[0],'major_category'] = major_category_table.loc[major_category_table['major_category'] == r[1][0]]['major_category_id'].index[0]
    crimes_df.at[r[0],'minor_category'] = minor_category_table.loc[minor_category_table['minor_category'] == r[1][1]]['minor_category_id'].index[0]

crimes_df = crimes_df.reset_index(drop=True)
crimes_df['crime_id'] = crimes_df.index 

In [42]:
crimes_df.head()

,major_category,minor_category,crime_id
0,0,0,0
1,1,1,1
2,0,2,2
3,0,3,3
4,0,4,4


#### Date Dataframe

In [43]:
date_df = crimes[['year', 'month']].drop_duplicates()

date_df.head()
date_df = date_df.reset_index(drop=True)
date_df['date_id'] = date_df.index

date_df.head()

,year,month,date_id
0,2013,7,0
1,2015,5,1
2,2013,8,2
3,2011,10,3
4,2016,4,4


In [44]:
crimes['crime_id'] = 'nan'
crimes['date_id'] = 'nan'
crimes['borough_id'] = 'nan'
crimes

,borough,major_category,minor_category,value,year,month,crime_id,date_id,borough_id
16,City of London,Violence Against the Person,Harassment,1,2013,7,nan,nan,nan
40,City of London,Violence Against the Person,Harassment,1,2015,5,nan,nan,nan
44,City of London,Violence Against the Person,Harassment,1,2013,8,nan,nan,nan
55,City of London,Violence Against the Person,Harassment,1,2011,10,nan,nan,nan
90,City of London,Violence Against the Person,Harassment,1,2016,4,nan,nan,nan
...,...,...,...,...,...,...,...,...,...
3886243,Westminster,Violence Against the Person,Wounding/GBH,87,2018,4,nan,nan,nan
3886244,Westminster,Violence Against the Person,Wounding/GBH,117,2018,5,nan,nan,nan
3886245,Westminster,Violence Against the Person,Wounding/GBH,101,2018,6,nan,nan,nan
3886246,Westminster,Violence Against the Person,Wounding/GBH,130,2018,7,nan,nan,nan


In [45]:
crimes = crimes[crimes['borough'] != 'City of London'] 

In [46]:
crimes

,borough,major_category,minor_category,value,year,month,crime_id,date_id,borough_id
1479,Ealing,Sexual Offences,Rape,1,2008,2,nan,nan,nan
1813,Ealing,Violence Against the Person,Murder,1,2011,5,nan,nan,nan
1819,Ealing,Violence Against the Person,Murder,1,2012,11,nan,nan,nan
1886,Ealing,Violence Against the Person,Murder,1,2008,9,nan,nan,nan
1904,Ealing,Violence Against the Person,Murder,1,2010,5,nan,nan,nan
...,...,...,...,...,...,...,...,...,...
3886243,Westminster,Violence Against the Person,Wounding/GBH,87,2018,4,nan,nan,nan
3886244,Westminster,Violence Against the Person,Wounding/GBH,117,2018,5,nan,nan,nan
3886245,Westminster,Violence Against the Person,Wounding/GBH,101,2018,6,nan,nan,nan
3886246,Westminster,Violence Against the Person,Wounding/GBH,130,2018,7,nan,nan,nan


In [47]:
#load demographics

demographics = pd.read_csv('D:/Universidade-Fcul/2_semestre/TPD/projeto/demographics.csv')
demographics.head()

,Area name,Inner/ Outer London,GLA Population Estimate 2016,GLA Household Estimate 2016,Inland Area (Hectares),Population density (per hectare) 2016,"Average Age, 2016","Proportion of population aged 0-15, 2016","Proportion of population of working-age, 2016","Proportion of population aged 65 and over, 2016",...,Anxiety score 2011-14 (out of 10),Childhood Obesity Prevalance (%) 2014/15,Mortality rate from causes considered preventable 2012/14,Political control in council,Proportion of seats won by Conservatives in 2014 election,Proportion of seats won by Labour in 2014 election,Proportion of seats won by Lib Dems in 2014 election,Turnout at 2014 local elections,borrough_id,borough_id
0,Barking and Dagenham,Outer London,205773.0,76841.0,3610.8,57.3,32.9,21.0,86.1,13.9,...,3.05,25.3,227.6,Lab,0.0,100.0,0.0,36.5,0,0
1,Barnet,Outer London,385108.0,149147.0,8674.8,44.5,37.2,21.0,83.3,16.7,...,2.75,18.4,133.8,Cons,50.8,47.6,1.6,40.5,1,1
2,Bexley,Outer London,243303.0,97233.0,6058.1,39.9,38.9,20.8,89.0,11.0,...,3.29,21.4,164.3,Cons,71.4,23.8,0.0,39.6,2,2
3,Brent,Outer London,328568.0,119166.0,4323.3,76.1,35.5,20.1,82.5,17.5,...,2.92,23.9,169.4,Lab,9.5,88.9,1.6,36.3,3,3
4,Bromley,Outer London,326560.0,139654.0,15013.5,21.7,40.1,15.8,88.4,11.6,...,3.26,16.5,148.5,Cons,85.0,11.7,0.0,40.8,4,4


In [118]:
demographics_aux = demographics[['Area name','borough_id']]
demographics_aux = demographics_aux.rename(columns = {'Area name': 'borough'})

In [86]:
crimes

,borough,major_category,minor_category,value,year,month,crime_id,date_id,borough_id
1479,Ealing,Sexual Offences,Rape,1,2008,2,21,51,7
1813,Ealing,Violence Against the Person,Murder,1,2011,5,nan,nan,nan
1819,Ealing,Violence Against the Person,Murder,1,2012,11,nan,nan,nan
1886,Ealing,Violence Against the Person,Murder,1,2008,9,nan,nan,nan
1904,Ealing,Violence Against the Person,Murder,1,2010,5,nan,nan,nan
...,...,...,...,...,...,...,...,...,...
3886243,Westminster,Violence Against the Person,Wounding/GBH,87,2018,4,nan,nan,nan
3886244,Westminster,Violence Against the Person,Wounding/GBH,117,2018,5,nan,nan,nan
3886245,Westminster,Violence Against the Person,Wounding/GBH,101,2018,6,nan,nan,nan
3886246,Westminster,Violence Against the Person,Wounding/GBH,130,2018,7,nan,nan,nan


In [68]:
major_category_table.loc[major_category_table['major_category'] == r[1][1]]['major_category_id'].index[0]
minor_category_table.loc[minor_category_table['minor_category'] == r[1][2]]['minore_category_id'].index[0]

21

In [79]:
crimes_df.loc[(crimes_df['major_category'] == major_category_table.loc[major_category_table['major_category'] == 'Sexual Offences']['major_category_id'].index[0])
                  & (crimes_df['minor_category'] == minor_category_table.loc[minor_category_table['minor_category'] == 'Rape']['minor_category_id'].index[0])].crime_id.index[0]

21

In [109]:
crimes_df.head()
crimes_df = crimes_df.rename(columns={"major_category": "major_category_id", "minor_category": "minor_category_id"})
crimes_df.head()

,major_category_id,minor_category_id,crime_id
0,0,0,0
1,1,1,1
2,0,2,2
3,0,3,3
4,0,4,4


In [ ]:
new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])
demographics_aux

In [119]:
aux = crimes.merge(major_category_table, on = "major_category")
aux = aux.merge(minor_category_table, on = "minor_category")
aux = aux.merge(crimes_df, on = ["major_category_id","minor_category_id"])
aux = aux.merge(date_df, on = ["year","month"])
aux = aux.merge(demographics_aux, on = ["borough"])
aux

,borough,major_category,minor_category,value,year,month,major_category_id,minor_category_id,crime_id,date_id,borough_id
0,Ealing,Violence Against the Person,Harassment,4,2013,7,0,0,0,0,7
1,Ealing,Violence Against the Person,Harassment,2,2013,7,0,0,0,0,7
2,Ealing,Violence Against the Person,Harassment,1,2013,7,0,0,0,0,7
3,Ealing,Violence Against the Person,Harassment,2,2013,7,0,0,0,0,7
4,Ealing,Violence Against the Person,Harassment,1,2013,7,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...
989887,Westminster,Criminal Damage,Criminal Damage To Other Bldg,34,2018,8,6,36,36,127,31
989888,Westminster,Criminal Damage,Criminal Damage To a Dwelling,22,2018,8,6,37,37,127,31
989889,Westminster,Sexual Offences,Rape,53,2018,8,7,21,21,127,31
989890,Westminster,Sexual Offences,Other Sexual,73,2018,8,7,24,24,127,31


In [123]:
fact_table_1 = aux.copy()
fact_table_1 = fact_table_1[['crime_id','date_id','borough_id','value']]
fact_table_1['ocurred_id'] = fact_table_1.index
fact_table_1.to_csv('C:/csvs/fact_table_small.csv', index=False)  

In [125]:
fact_table_2 = aux.copy()
fact_table_2 = fact_table_2[['major_category','minor_category','value','year','month','borough_id']]
fact_table_2['ocurred_id'] = fact_table_2.index
fact_table_2
fact_table_2.to_csv('C:/csvs/fact_table_big.csv', index=False)  

In [126]:
fact_table_1

,crime_id,date_id,borough_id,value,ocurred_id
0,0,0,7,4,0
1,0,0,7,2,1
2,0,0,7,1,2
3,0,0,7,2,3
4,0,0,7,1,4
...,...,...,...,...,...
989887,36,127,31,34,989887
989888,37,127,31,22,989888
989889,21,127,31,53,989889
989890,24,127,31,73,989890


In [1]:
for r in crimes.iterrows():
    #Linha feia
    crimes.at[r[0],'crime_id'] = crimes_df.loc[(crimes_df['major_category'] == major_category_table.loc[major_category_table['major_category'] == r[1][1]]['major_category_id'].index[0])
                                  & (crimes_df['minor_category'] == minor_category_table.loc[minor_category_table['minor_category'] == r[1][2]]['minor_category_id'].index[0])].crime_id.index[0]
    
    crimes.at[r[0],'date_id'] = date_df.loc[(date_df['year'] == r[1][4]) & (date_df['month'] == r[1][5])].index[0]
    crimes.at[r[0],'borough_id'] = demographics.loc[(demographics['Area name'] == r[1][0])]['borough_id'].values[0]
#     break

In [73]:
crimes_df.loc[crimes_df['crime_id'] != 'nan']

,major_category,minor_category,crime_id
0,0,0,0
1,1,1,1
2,0,2,2
3,0,3,3
4,0,4,4
5,2,5,5
6,0,6,6
7,3,7,7
8,1,8,8
9,4,9,9


In [21]:
ocurred = crimes[['crime_id','date_id','borough_id','value']]
ocurred = ocurred.reset_index(drop = True)
ocurred['ocurred_id'] = ocurred.index
ocurred.head()

#### Export tables to csv

In [29]:
major_category_table.head()

,major_category,major_category_id
0,Violence Against the Person,0
1,Theft and Handling,1
2,Drugs,2
3,Other Notifiable Offences,3
4,Robbery,4


In [30]:
minor_category_table.head()

,minor_category,minor_category_id
0,Harassment,0
1,Other Theft,1
2,Wounding/GBH,2
3,Common Assault,3
4,Other violence,4


In [34]:
crimes_df.head()

,major_category,minor_category,crime_id
0,0,0,0
1,1,1,1
2,0,2,2
3,0,3,3
4,0,4,4


In [35]:
ocurred.head()

,crime_id,date_id,borough_id,value,ocurred_id
0,21,51,7,1,0
1,22,52,7,1,1
2,22,22,7,1,2
3,22,53,7,1,3
4,22,54,7,1,4


In [37]:
# compression_opts = dict(method='zip', archive_name='crimes_merged.csv')

# major_category_table.to_csv('major.zip', compression=compression_opts, index=False)
# minor_category_table.to_csv('minor.zip', compression=compression_opts, index=False)
# crimes_df.to_csv('crimes.zip', compression=compression_opts, index=False)
# ocurred.to_csv('ocurred.zip', compression=compression_opts, index=False)

In [40]:
demographics.to_csv('demographics.zip', compression=compression_opts, index=False)

In [14]:
dates = crimes[['year','month']].reset_index(drop = 1)

In [18]:
dates = dates.drop_duplicates().reset_index(drop = 1)

In [20]:
dates['id'] = dates.index

In [21]:
dates

,year,month,id
0,2013,7,0
1,2015,5,1
2,2013,8,2
3,2011,10,3
4,2016,4,4
...,...,...,...
123,2018,4,123
124,2018,5,124
125,2018,6,125
126,2018,7,126


In [37]:
compression_opts = dict(method='zip', archive_name='date.csv')
date_df.to_csv('date.zip', compression=compression_opts, index=False)